In [ ]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import random
from tensorflow import keras
from tensorflow.keras import layers
import scipy

In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("GPU Devices:", tf.config.list_physical_devices('GPU'))


In [ ]:
img_array=cv2.imread("Training/2/Training_135069.jpg")
img_array.shape

In [ ]:
plt.imshow(img_array)

In [ ]:
Datadir = "Training/"
classes = ["0","1","2","3","4","5","6"]

In [ ]:
for category in classes:
    path=os.path.join(Datadir, category)
    for img in os.listdir(path):
        img_array=cv2.imread(os.path.join(path,img))
        plt.imshow(cv2.cvtColor(img_array,cv2.COLOR_BGR2RGB))
        plt.show()
        break
    break

In [ ]:
img_size = 224
new_array = cv2.resize(img_array, (img_size,img_size))
plt.imshow(cv2.cvtColor(new_array, cv2.COLOR_BGR2RGB))
plt.show()


In [ ]:
new_array.shape

In [ ]:
training_Data = []
def create_training_data():
    for category in classes:
        path=os.path.join(Datadir, category)
        class_num = classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(img_size,img_size))
                training_Data.append([new_array,class_num])
            except Exception as e:
                pass

In [ ]:
create_training_data()
print(len(training_Data))


In [ ]:
random.shuffle(training_Data)

In [ ]:
X = []
y = []

for features,label in training_Data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, img_size, img_size, 3)

In [ ]:
X.shape

In [ ]:
#X = X.astype('float32') / 255.0

In [ ]:
import numpy as np

# Suppose X is your original uint8 array
X= np.array([img.astype('float32')/255.0 for img in X], dtype='float32')


In [ ]:
Y=np.array(y)

In [ ]:
model=tf.keras.applications.MobileNetV2()

In [ ]:
model.summary()

In [ ]:
base_input = model.layers[0].input

In [ ]:
base_output=model.layers[-2].output

In [ ]:
base_output

In [ ]:
base_input

In [ ]:
final_output = layers.Dense(128)(base_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(64)(final_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(7,activation='softmax')(final_output)

In [ ]:
final_output

In [ ]:
new_model = keras.Model(inputs = base_input, outputs = final_output)

In [ ]:
new_model.summary()

In [ ]:
new_model.compile(loss="sparse_categorical_crossentropy", optimizer = "adam" , metrics = ["accuracy"])

In [ ]:
#new_model.fit(X,Y,epochs=25)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1/255.0)

batch_size = 32

train_generator = datagen.flow(X, Y, batch_size=batch_size)

new_model.fit(train_generator, epochs=20)


In [ ]:
new_model.save('final_Model-95p07.h5')

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('25EP.h5')

print("Input shape:", model.input_shape)
print("Output shape:", model.output_shape)
model.summary()


In [ ]:
frame = cv2.imread("11.jpg")

In [ ]:
frame.shape

In [ ]:
faceCascade=cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

In [ ]:
gray.shape

In [ ]:
faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
for x,y,w,h in faces:
    roi_gray = gray[y:y+h,x:x+w]
    roi_color = frame[y:y+h,x:x+w]
    cv2.rectangle(frame,(x,y), (x+w , y+h), (255,0,0),2)
    faces= faceCascade.detectMultiScale(roi_gray)
    if len(faces)==0:
        print("Face not detected")
    else:
        for(ex,ey,ew,eh)in faces:
            face_roi = roi_color[ey: ey+eh, ex:ex + ew]

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [ ]:
plt.imshow(cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB))

In [ ]:
final_image = cv2.resize(face_roi, (224,224))
final_image = np.expand_dims(final_image,axis = 0)
final_image=final_image/255.0

In [ ]:
from tensorflow import keras

# Load the model
model = keras.models.load_model("20EP.h5")
Predicitions=model.predict(final_image)

In [ ]:
Predicitions[0]

In [ ]:
np.argmax(Predicitions)